In [1]:
import yaml
import glob
import pandas as pd
import numpy as np
import os
import argparse
import pickle
import sys
sys.path.append('./../..')
sys.path.append('./..')

try:
    from .metapath2vec_g import metapath2vec_g_model
except:
    from metapath2vec_g import metapath2vec_g_model

try:
    from src.data_fetcher import data_fetcher_v2 as data_fetcher
except:
    from .src.data_fetcher import data_fetcher_v2 as data_fetcher
    
from gensim.similarities.index import AnnoyIndexer
from gensim.models.word2vec import LineSentence
from gensim.models import Word2Vec

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
coOccDict_file = './model_use_data/us_import2/coOccMatrixSaved.pkl'
with open(coOccDict_file, 'rb') as fh:
    coOccDict = pickle.load(fh)

In [2]:
model_save_path = './model_weights_data/metapath2vec_gensim/us_import2/mp2v_gensim.data'
model = Word2Vec.load(model_save_path)

In [6]:
 annoy_index = AnnoyIndexer(model, 128)


/home/ddatta/anaconda3/envs/AD_v2/lib/python3.6/site-packages/gensim/similarities/index.py:184: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  index = AnnoyIndex(num_features)


In [8]:
serial_mapping_df = pd.read_csv('./model_use_data/us_import2/Serialized_Mapping.csv',index_col=None)

In [9]:
serial_mapping_df

,Domain,Entity_ID,Serial_ID
0,Carrier,0,0
1,Carrier,1,1
2,Carrier,2,2
3,Carrier,3,3
4,Carrier,4,4
...,...,...,...
8935,ShipperPanjivaID,4078,8935
8936,ShipperPanjivaID,4079,8936
8937,ShipperPanjivaID,4080,8937
8938,ShipperPanjivaID,4081,8938


In [25]:
PortOfLadingSamples =serial_mapping_df.loc[serial_mapping_df['Domain']=='PortOfLading'].sample(20)
del PortOfLadingSamples['Domain']
PortOfLadingSamples

,Entity_ID,Serial_ID
4425,16,4425
4464,55,4464
4546,137,4546
4545,136,4545
4587,178,4587
4554,145,4554
4431,22,4431
4579,170,4579
4555,146,4555
4577,168,4577


In [17]:
def get_Entity_ID(serial_ID):
    global serial_mapping_df
    d = list(serial_mapping_df.loc[
                 (serial_mapping_df['Serial_ID'] == serial_ID)
             ]['Domain'])[0]
    e = list(serial_mapping_df.loc[
                 (serial_mapping_df['Serial_ID'] == serial_ID)
             ]['Entity_ID'])[0]
    return e, d

In [28]:
dom_list = ['PortOfLading' , 'PortOfUnLading', 'ShipmentOrigin', 'ShipmentDestination', 'HSCode']
for dom in dom_list:
    print(" ================================= ")
    samples = serial_mapping_df.loc[serial_mapping_df['Domain']== dom ].sample(20)
    del PortOfLadingSamples['Domain']
    
    for i,row in PortOfLadingSamples.iterrows():
        print( '---------------------')

        s_id = row['Serial_ID']
        vector = model.wv[str(s_id)]
        approximate_neighbors = model.wv.most_similar(
                [vector],
                topn=25,
                indexer=annoy_index
            )
        e1 , _ = get_Entity_ID(s_id)
        for n_s_id in approximate_neighbors[1:]:
                n_s_id = int(n_s_id[0])
                e, d = get_Entity_ID(n_s_id)
                if dom == d : continue
                if d < dom:
                    key = d + '_+_' + dom
                    align = 'r'
                else:
                    key = dom + '_+_' + d
                    align = 'c'

                matrix = coOccDict[key]
                if align == 'r':
                    arr = matrix[e, :]
                else:
                    arr = matrix[:, e]
                print('Domain ', d, 'Entity ', e,' Count', arr[e1])

 >>> Entity_ID      16
Serial_ID    4425
Name: 4425, dtype: int64
 >>> Entity_ID      55
Serial_ID    4464
Name: 4464, dtype: int64
Domain  ShipperPanjivaID Entity  4080  Count 115.0
 >>> Entity_ID     137
Serial_ID    4546
Name: 4546, dtype: int64
Domain  ShipperPanjivaID Entity  391  Count 2.0
Domain  Carrier Entity  307  Count 2.0
Domain  ShipperPanjivaID Entity  2518  Count 0.0
Domain  ShipperPanjivaID Entity  422  Count 0.0
Domain  ShipperPanjivaID Entity  2203  Count 0.0
Domain  ShipperPanjivaID Entity  1102  Count 0.0
Domain  ShipperPanjivaID Entity  878  Count 0.0
Domain  ShipperPanjivaID Entity  632  Count 0.0
Domain  ShipperPanjivaID Entity  2910  Count 0.0
Domain  ShipperPanjivaID Entity  1219  Count 0.0
Domain  ShipperPanjivaID Entity  2906  Count 0.0
Domain  ShipperPanjivaID Entity  888  Count 0.0
Domain  ShipperPanjivaID Entity  3136  Count 0.0
Domain  ShipperPanjivaID Entity  803  Count 0.0
Domain  ShipperPanjivaID Entity  407  Count 0.0
Domain  ShipperPanjivaID Entity  